In [ ]:
import tensorflow as tf
from tensorflow import keras
import h5py
import scipy.stats
import numpy as np
import os
import pandas as pd
from sklearn import model_selection
os.environ['CUDA_VISIBLE_DEVICES'] = '0'
celltype = 'HepG2'

In [ ]:
model = keras.models.load_model('/home/ztang/multitask_RNA/model/lenti_MPRA_onehot/'+celltype+'/ResNet.h5')

In [ ]:
f = h5py.File('/home/ztang/multitask_RNA/data/lenti_MPRA/'+celltype+'_onehot.h5', 'r')
x = f['onehot'][()]
y = f['target'][()]
x = np.swapaxes(x,1,2)
x_train, x_test, y_train, y_test = model_selection.train_test_split(x, y, test_size=0.1,random_state=42)
model.evaluate(x_test,y_test)

In [ ]:
## Test set performance
y_pred = model.predict(x_test)
scipy.stats.pearsonr(np.squeeze(y_pred),y_test)

In [ ]:
file = h5py.File("/home/ztang/multitask_RNA/data/CAGI/"+celltype+"/onehot.h5", "r")
alt = file['alt']
ref = file['ref']
alt_pred = model.predict(alt)
ref_pred = model.predict(ref)
pred = alt_pred - ref_pred
pred.shape

In [ ]:
exp_df = pd.read_csv('/home/ztang/multitask_RNA/data/CAGI/'+celltype+'/metadata.csv')
target = exp_df['6'].values.tolist()

In [ ]:
#CAGI performance
start_idx=0
perf = []
for exp in exp_df['8'].unique():
    sub_df = exp_df[exp_df['8'] == exp]
    exp_target = np.array(target)[sub_df.index.to_list()]
    exp_pred = np.squeeze(pred)[sub_df.index.to_list()]
    print(exp)
    perf.append(scipy.stats.pearsonr(exp_pred,exp_target)[0])
    print(scipy.stats.pearsonr(exp_pred,exp_target)[0])

## All CAGI performance

In [ ]:
import tensorflow as tf
from tensorflow import keras
import h5py
import scipy.stats
import numpy as np
import os
import pandas as pd
from sklearn import model_selection
os.environ['CUDA_VISIBLE_DEVICES'] = '0'
celltype = 'HepG2'

In [ ]:
model = keras.models.load_model('/home/ztang/multitask_RNA/model/lenti_MPRA_onehot/'+celltype+'/model.h5')

In [ ]:
file = h5py.File("/home/ztang/multitask_RNA/data/CAGI/230/CAGI_onehot.h5", "r")
alt = file['alt']
ref = file['ref']
alt_pred = model.predict(alt)
ref_pred = model.predict(ref)
pred = alt_pred - ref_pred
pred.shape

In [ ]:
cagi_df = pd.read_csv('../../data/CAGI/230/final_cagi_metadata.csv',
                      index_col=0).reset_index()
exp_list = cagi_df['8'].unique()
target = cagi_df['6'].values.tolist()

In [ ]:
import scipy.stats as stats
perf = []
sanity_check = 0
for exp in cagi_df['8'].unique():
    sub_df = cagi_df[cagi_df['8'] == exp]
    sanity_check += len(sub_df)
    exp_target = np.array(target)[sub_df.index.to_list()]
    exp_pred = np.squeeze(pred)[sub_df.index.to_list()]
    print(exp)
    perf.append(stats.pearsonr(exp_pred,exp_target)[0])
    print(stats.pearsonr(exp_pred,exp_target)[0])

In [ ]:
np.mean(perf)